In [ ]:
from collections import defaultdict
import numpy as np
import scipy
import scipy.sparse as sps
import math
import matplotlib.pyplot as plt
import time
from sklearn.datasets import load_svmlight_file
from helpers import *
import random
%matplotlib inline

# Lasso with SGD and Coordinate Descent

The optimization problem for sparse least squares linear regression (also known as the Lasso) is given by
\begin{equation}\label{eq:lasso}
  \min_{x \in R^n} \   \|Ax - b\|^2 + \lambda \|x\|_1
\end{equation}
for some regularization parameter $\lambda>0$, where $A\in R^{d\times n}$ is the given data matrix, and $b\in R^d$ is the given ``measurement'' vector. The $\|.\|_1$-norm is the sum of absolute values of coordinates of its argument.

The goal of this exercise is to implement stochastic (sub)gradient descent (SGD) as well as coordinate descent on this objective function, where both algorithms will work based on the $x$ variable vector.


# Data

In [ ]:
d = 11
A = np.random.normal(size = (d, 10))
b = np.random.uniform(1, 0, d)

or use a real dataset in LibSVM format

In [ ]:
A, b = load_svmlight_file("data/ionosphere.txt")

# Stochastic gradient descent for Lasso

In [ ]:
def lasso_function(A, b, reg_coef, alpha):
    return 0.5 * np.linalg.norm(A.dot(alpha) - b) ** 2 + reg_coef * np.sum(np.abs(alpha))

In [ ]:
def compute_stoch_gradient_lasso(A, b, reg_coef, alpha):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    err = b - A.dot(alpha)
    gradient = - A.T.dot(err) + reg_coef * np.sign(alpha) * A.shape[0]
    return gradient

In [ ]:
def stochastic_gradient_descent_lasso(A, b, reg_coef, gamma, batch_size=1, max_iter=1000, trace=False):
    history = defaultdict(list) if trace else None
    
    num_data_points, num_features = np.shape(A)
    alpha_t = np.zeros(num_features)
    
    start_time = time.time()
    
    for current_iter in range(0, max_iter):
        if trace:
            history['time'].append(time.time() - start_time)
            history['objective_function'].append(lasso_function(A, b, reg_coef, alpha_t))
        ...
        
    return alpha_t, history

In [ ]:
x_t, history = stochastic_gradient_descent_lasso(A, b, 0.1, 0.005, max_iter=5000, trace=True, batch_size=1)

In [ ]:
plt.semilogy(history['objective_function']) # log scale
plt.xlabel("iteration")
plt.ylabel("objective value")
plt.title("objective value")

# Coordinate Descent for Lasso

In [ ]:
def soft_threshold(internal, reg_coef, current_feature_norm):
    if internal > reg_coef:
        return (internal - reg_coef) / (current_feature_norm ** 2)
    elif internal < - reg_coef:
        return (internal + reg_coef) / (current_feature_norm ** 2)
    return 0.0

In [ ]:
def coordinate_descent_lasso(A, b, reg_coef, max_iter=1000, trace=False, is_check=False,
                             check_epsilon=1e-1):
    history = defaultdict(list) if trace else None

    num_features = np.shape(A)[1]
    alpha_t = np.zeros(num_features)
    residual = np.copy(b)

    start_time = time.time()

    for current_iter in range(0, max_iter):
        if trace:
            history['time'].append(time.time() - start_time)
            history['objective_function'].append(lasso_function(A, b, reg_coef, alpha_t))
        i = np.random.randint(0, num_features)
        if sps.issparse(A):
            current_feature = np.array(A[:, i].todense())
        else:
            current_feature = np.array(A[:, i])

        current_feature_norm = np.linalg.norm(current_feature)
        if current_feature_norm == 0:
            alpha_t[i] = 0
            continue
        
        ...

        new_value = soft_threshold( ... , reg_coef, current_feature_norm)
        residual += (current_feature * (alpha_t[i] - new_value)).reshape(residual.shape)
        alpha_t[i] = np.copy(new_value)

        if is_check:  # implement an optional check if a slightly smaller or larger step on 
                      # the current coordinate would indeed make the function value worse
            
    return alpha_t, residual, history

And finally run the optimization algorithm

In [ ]:
x_t, residual, history = coordinate_descent_lasso(A, b, 0.1, max_iter=1000, trace=True,
                                                  is_check=True, check_epsilon=1e-6)

In [ ]:
plt.semilogy(history['objective_function'])
plt.xlabel("iteration")
plt.ylabel("objective value")
plt.title("objective value")

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1 of the ML course, available from https://inclass.kaggle.com/c/epfml-project-1/data

The original optimization problem for the Support Vector Machine (SVM) is given by
\begin{equation}\label{eq:primal}
  \min_{w \in R^d} \  \sum_{i=1}^n \ell(y_i A_i^\top w) + \frac\lambda2 \|w\|^2
\end{equation}
where $\ell : R\rightarrow R$, $\ell(z) := \max\{0,1-z\}$ is the hinge loss function.
Here for any $i$, $1\le i\le n$, the vector $A_i\in R^d$ is the $i$-th data example, and $y_i\in\{\pm1\}$ is the corresponding label.
  
The dual optimization problem for the SVM is given by 
\begin{equation}\label{eq:dual}
 \max_{\boldsymbol{\alpha} \in R^n } \  \alpha^\top\boldsymbol{1} - \tfrac1{2\lambda} \alpha^\top Y A^\top AY\alpha
 \text{    such that    $0\le \alpha_i \le 1  \ \forall i$}
\end{equation}
where $Y := \mathop{diag}(y)$, and $A\in R^{d\times n}$ again collects all $n$ data examples as its columns. 



In [ ]:
DATA_TRAIN_PATH = 'data/train.csv'

y, A, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, A.shape)

## Prepare cost and prediction functions

In [ ]:
def calculate_primal_objective(y, A, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    A: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    raise NotImplementedError

In [ ]:
def calculate_accuracy(y, A, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    A: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    raise NotImplementedError

## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the i-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [ ]:
def calculate_coordinate_update(y, A, lambda_, alpha, w, i):
    """compute a coordinate update (closed form) for coordinate i.
    A: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_examples)
    n: the coordinate to be updated
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # calculate the update of coordinate at index=n.
    a_i, y_i = A[i], y[i]
    old_alpha_i = np.copy(alpha[i])
    
    raise NotImplementedError
    return w, alpha

In [ ]:
def calculate_dual_objective(y, A, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    raise NotImplementedError

In [ ]:
def coordinate_descent_for_svm_demo(y, A):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = A.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # i = sample one data point uniformly at random from the columns of A
        i = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, A, lambda_, alpha, w, i)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, A, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, A, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, A, w)))

coordinate_descent_for_svm_demo(y, A)